In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
# read csv
entiredf=pd.read_csv("diabetes.csv")
entiredf.dtypes
df=entiredf

In [2]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [3]:
#1. Find total record for class 0 and class 1 from the outcome column
print("Total record for class 0 :",df.Outcome[df.Outcome==0].count() ,"\nTotal record for class 1 :",df.Outcome[df.Outcome==1].count())

Total record for class 0 : 500 
Total record for class 1 : 268


In [4]:
#2. Find Distinct values for different attributes (Columns) and print total count for each.
print("The attributes of the data frame are :",list(df.columns))
while 1:
    attr=str(input("\nEnter the required attribute :"))
    if attr not in df.columns:
        print("Incorect attribute!! \nPOSSIBLE VALS \n",list(df.columns))
    else:
        print("\n\nUnique values and counts")
        print(df.groupby([attr]).Outcome.count().to_frame('count')) # to create count column heading used conversion to df
        break
        

if 1==0 :
    for i in df.columns:
        print(df.groupby([i]).Outcome.count())
        print("\n\n\n")



The attributes of the data frame are : ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

Enter the required attribute :Pregnancies


Unique values and counts
             count
Pregnancies       
0              111
1              135
2              103
3               75
4               68
5               57
6               50
7               45
8               38
9               28
10              24
11              11
12               9
13              10
14               2
15               1
17               1


In [5]:
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
#3. What is the Gini coefficient of the attributes(Columns).

def gini(col):
    #https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurit
    #https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb
    #https://www.youtube.com/watch?v=7VeUPuFGJHk&t=499s       stack quest
    # information gain... confusion!!!!
    
    if not(isinstance(col,pd.core.series.Series)):
        print("pass pandas series as argument (pd.core.series.Series) ")
        return None
      
    impurity = 1.0
    no_of_entrys=float(len(col))
    vals_count_dict=dict(col.value_counts()) # converted value_counts to a dictionary
    for key in vals_count_dict :
         key_probablity=vals_count_dict[key] / no_of_entrys # no of occurance of an element by / total of occurances of elements
         impurity -= key_probablity ** 2  #  G = 1 - Sigma  Pi*Pi  for  i   0 to  no_of_types_of_elements
    return impurity


#  split branches takes column to be splitted and outcome cols    while returning left true and right
def split_branches(col,outcome):
    return col[outcome==1],col[outcome==0] # return true branch left and false at right




# to find total gini impurity __ using left and right branches w.r.t outcome __
#  using the weights also 
#  (no_of_Left_row / total rows)(left_Gini) * (    (no_of_Right_row / total rows)(Right_Gini)  ) ------->eqn: 1

def total_gini(df,col="NONE",Outcome_at=-1):
    if col == "NONE ":
        print("Enter column name as second arg ")
        return None
    left_branch,right_branch = split_branches(df[col],df.iloc[:,Outcome_at])
    # now calculate gini left and gini right 
    gini_left,gini_right = gini(left_branch), gini(right_branch)
    # Then calculate total gini  as in eqn: 1  
    Gini_total =  ( ( len(left_branch) /(  (len(left_branch) + len (right_branch))  ) ) * gini_left ) \
                 +( ( len(right_branch) /( (len(left_branch) + len (right_branch))  ) )* gini_right )
    return Gini_total
    
    
    
    

In [7]:
print(list(df.columns))
for i in df.iloc[:,:-1].columns:
    print(i,"  ",total_gini(df,i))
total_gini(df,col="Pregnancies")

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
Pregnancies    0.8879085043532338
Glucose    0.9866325404228844
BloodPressure    0.9545571361940293
SkinThickness    0.8960757929104473
Insulin    0.7591714863184059
BMI    0.9919699160447737
DiabetesPedigreeFunction    0.996096393034821
Age    0.9524200093283581


0.8879085043532338

In [8]:
#4. Find DiabetesPedigreeFunction which is < .200 if skin thickness and Insulin is o .
df.DiabetesPedigreeFunction[(df.DiabetesPedigreeFunction < 0.200)  & (df.SkinThickness==0) & (df.Insulin == 0) ]

7      0.134
10     0.191
33     0.188
81     0.102
101    0.179
123    0.186
154    0.137
170    0.180
178    0.190
180    0.084
190    0.142
196    0.187
207    0.151
226    0.198
268    0.078
283    0.165
319    0.129
333    0.137
347    0.187
400    0.161
401    0.151
404    0.135
430    0.108
433    0.167
468    0.183
474    0.126
494    0.174
518    0.180
522    0.189
524    0.151
536    0.197
560    0.121
583    0.190
586    0.129
596    0.194
598    0.088
601    0.190
616    0.157
619    0.141
624    0.158
628    0.144
634    0.167
636    0.153
642    0.178
658    0.190
660    0.182
678    0.127
694    0.191
708    0.148
714    0.121
729    0.141
749    0.178
758    0.197
762    0.142
Name: DiabetesPedigreeFunction, dtype: float64

In [9]:
#5. Check what will be the blood presser level if pregnancies count is 10 and print the average BP level.
df.loc[:,['Pregnancies','BloodPressure']]
print("average blood pressure :",df.BloodPressure.mean())

# two variable for regression


7        0
11      74
12      80
25      70
34      78
143     66
246     68
270     86
281     76
306     68
327     70
458     84
464     98
505     82
542     85
578     68
634     62
660     84
667     70
672    106
706      0
712     62
717     72
763     76
Name: BloodPressure, dtype: int64
average blood pressure : 69.10546875
